In [153]:
#Prettify Json

import json

with open('Primi_1.json', 'r') as file:
    json_data = json.load(file)

pretty_json = json.dumps(json_data, indent=4)

with open('Primi_1.json', 'w') as file:
    file.write(pretty_json)

In [154]:
#Get Indexing positions

import pickle
with open("depth_to_global_primi_1.pkl", 'rb') as f:
    depth_to_global = pickle.load(f)

global_to_depth = {v: k for k, v in depth_to_global.items()}

global_to_depth

{0: (3, 0),
 1: (0, 1),
 2: (1, 2),
 3: (3, 3),
 4: (0, 4),
 5: (0, 5),
 6: (0, 6),
 7: (0, 7),
 8: (0, 8),
 9: (3, 9)}

In [155]:
#Extract depth

with open("Primi_1_full_graph.pkl", 'rb') as f:
    data = pickle.load(f)

data = data['data']

i = 0
treble_depths = []
bass_depths = []
while True:
    try:
        treble_depths.append(sorted(list(set((data[('note', f'treble_depth{i}', 'note')].edge_index).flatten().tolist()))))
        bass_depths.append(sorted(list(set((data[('note', f'bass_depth{i}', 'note')].edge_index).flatten().tolist()))))
        i += 1
    except AttributeError:
        break

idx_depth = {}

for i in range(len(treble_depths)):
    for j in treble_depths[i]:
        idx_depth[j] = i
    for k in bass_depths[i]:
        if i == 0 and k in idx_depth:
            idx_depth[k+0.1] = i
        elif k+0.1 in idx_depth:
            idx_depth[k+0.1] = i
        else:
            idx_depth[k] = i

for key,v in global_to_depth.items():
    if key not in idx_depth:
        idx_depth[key] = 0

In [156]:
#Reposition notes global_to_depth

repositioned_global_to_depth = {}

for i in treble_depths[0]:
    repositioned_global_to_depth[i] = (0, global_to_depth[i][1])

for j in bass_depths[0]:
    if j+0.1 in idx_depth:
        repositioned_global_to_depth[j+0.1] = (3, global_to_depth[j][1])
    else:
        repositioned_global_to_depth[j] = (3, global_to_depth[j][1])

for k, v in global_to_depth.items():
    if k not in repositioned_global_to_depth:
        repositioned_global_to_depth[k] = v

repositioned_global_to_depth

{1: (0, 1),
 4: (0, 4),
 5: (0, 5),
 6: (0, 6),
 7: (0, 7),
 8: (0, 8),
 9: (0, 9),
 0: (3, 0),
 3: (3, 3),
 9.1: (3, 9),
 2: (1, 2)}

In [157]:
#Get pitch for each note

with open('Primi_1.json', 'r') as file:
    json_file = json.load(file)

t_pitch = json_file['trebleNotes']['pitchNames']
b_pitch = json_file['bassNotes']['pitchNames']
it_pitch = json_file['innerTrebleNotes']['pitchNames']
ib_pitch = json_file['innerBassNotes']['pitchNames']

pitches = [t_pitch, it_pitch, ib_pitch, b_pitch]

idx_pitch = {}

for idx, position in global_to_depth.items():
    idx_pitch[idx] = pitches[position[0]][position[1]]

In [158]:
#Add in pitch and depth for repositioned notes

verticalities = len(t_pitch)
new_t_pitch = ['_'] * verticalities
new_b_pitch = ['_'] * verticalities
new_it_pitch = ['_'] * verticalities
new_ib_pitch = ['_'] * verticalities

new_pitch = [new_t_pitch, new_it_pitch, new_ib_pitch, new_b_pitch]
new_depth = [[0] * verticalities for _ in range(4)]

for idx, position in repositioned_global_to_depth.items():
    try:
        new_pitch[position[0]][position[1]] = idx_pitch[idx]
    except KeyError:
        new_pitch[position[0]][position[1]] = idx_pitch[idx - 0.1]
    new_depth[position[0]][position[1]] = idx_depth[idx]

new_depth

[[0, 3, 0, 0, 0, 2, 0, 1, 0, 3],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 2, 0, 0, 0, 0, 0, 3]]

In [159]:
#Shove it back to json

voice = ['trebleNotes', 'innerTrebleNotes', 'innerBassNotes', 'bassNotes']
for i in range(len(voice)):
    json_file[voice[i]]['pitchNames'] = new_pitch[i]
    json_file[voice[i]]['depths'] = new_depth[i]

In [160]:
pretty_json = json.dumps(json_file, indent=4)

with open('Primi_1_inferred.json', 'w') as file:
    file.write(pretty_json)